# Volume 2, Chapter 13: Network Documentation Basics

**AI-Powered Network Configuration Documentation**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/master/Volume-2-Practical-Applications/Colab-Notebooks/Vol2_Ch13_API_Builder.ipynb)

## What You'll Learn

This notebook demonstrates:
- ✅ Auto-generate network documentation from device configs
- ✅ Extract structured data (interfaces, routing, security)
- ✅ Create network topology diagrams (Mermaid format)
- ✅ Build automated documentation pipelines
- ✅ Version control documentation with Git
- ✅ Deploy to production with CI/CD

By the end, you'll have practical code you can use on your own network.

## Setup & Dependencies

In [ ]:
# Install required packages
!pip install -q anthropic python-dotenv gitpython schedule

import os
import json
from datetime import datetime
from pathlib import Path
from typing import Dict, List
from getpass import getpass

# Configure API key
if 'ANTHROPIC_API_KEY' not in os.environ:
    api_key = getpass('Enter your Anthropic API key: ')
    os.environ['ANTHROPIC_API_KEY'] = api_key
else:
    api_key = os.environ['ANTHROPIC_API_KEY']

print("✅ Dependencies installed and API key configured!")

## The Problem We're Solving

### Why Network Documentation Fails

**Scenario**: Your company has 500 network devices. Documentation exists in:
- SharePoint docs (outdated, conflicting)
- Wiki pages (incomplete, different formats)
- Tribal knowledge (in one person's head)
- Running configs (the actual source of truth)
- Nowhere (most of the time)

**The Problem**:
- New engineer spends 3-5 days learning the network
- Changes made without understanding implications
- Configuration drift goes unnoticed
- Audits find undocumented devices
- When someone leaves, knowledge disappears

**The Solution**: Generate documentation automatically from device configs using AI.

## Part 1: Documentation Generation

Auto-generate complete device documentation from a network configuration.

In [ ]:
from anthropic import Anthropic

class ConfigDocumentationGenerator:
    """Generate documentation automatically from network configs."""

    def __init__(self, api_key: str):
        self.client = Anthropic(api_key=api_key)
        self.usage_stats = {"input_tokens": 0, "output_tokens": 0}

    def generate_device_overview(self, config: str, hostname: str) -> Dict:
        """Generate high-level device documentation."""

        prompt = f"""Analyze this network device configuration and create documentation.

Device: {hostname}
Configuration:
{config}

Extract and document:
1. Device role (core router, access switch, firewall, etc.)
2. Management IP address
3. Routing protocols in use (BGP, OSPF, EIGRP, static)
4. Key features enabled (HSRP, VRF, QoS, etc.)
5. Interface count and types
6. Notable configurations or policies

Return as JSON:
{{
    "hostname": "device name",
    "role": "device role",
    "management_ip": "IP address",
    "routing_protocols": ["list of protocols"],
    "key_features": ["list of features"],
    "interface_summary": "summary of interfaces",
    "notable_config": "anything important to know"
}}

JSON:"""

        response = self.client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1500,
            temperature=0,
            messages=[{"role": "user", "content": prompt}]
        )

        # Track usage
        self.usage_stats["input_tokens"] += response.usage.input_tokens
        self.usage_stats["output_tokens"] += response.usage.output_tokens

        doc_data = json.loads(response.content[0].text)
        doc_data['generated_at'] = datetime.now().isoformat()

        return doc_data

    def generate_interface_table(self, config: str) -> str:
        """Generate markdown table of all interfaces."""

        prompt = f"""Extract all interfaces from this config and create a markdown table.

Config:
{config}

Create a table with columns:
| Interface | IP Address | Status | Description | VLAN/VRF |

Include ALL interfaces (physical, loopback, tunnel, etc.)

Markdown table:"""

        response = self.client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=2000,
            temperature=0,
            messages=[{"role": "user", "content": prompt}]
        )

        # Track usage
        self.usage_stats["input_tokens"] += response.usage.input_tokens
        self.usage_stats["output_tokens"] += response.usage.output_tokens

        return response.content[0].text

    def generate_routing_documentation(self, config: str) -> str:
        """Document routing configuration."""

        prompt = f"""Document the routing configuration from this device.

Config:
{config}

Create documentation covering:

## Routing Protocols
- Which protocols are enabled
- Process IDs, AS numbers
- Router IDs

## Static Routes
- Destination networks
- Next hops
- Purpose

## Route Redistribution
- What's redistributed where
- Filters applied

## Routing Policies
- Route-maps
- Prefix-lists
- Access-lists affecting routing

Format as markdown with sections and bullet points.

Documentation:"""

        response = self.client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=2000,
            temperature=0,
            messages=[{"role": "user", "content": prompt}]
        )

        # Track usage
        self.usage_stats["input_tokens"] += response.usage.input_tokens
        self.usage_stats["output_tokens"] += response.usage.output_tokens

        return response.content[0].text

    def generate_complete_documentation(
        self,
        config: str,
        hostname: str
    ) -> str:
        """Generate complete device documentation."""

        print(f"🔄 Generating documentation for {hostname}...")

        # Get all sections
        overview = self.generate_device_overview(config, hostname)
        interfaces = self.generate_interface_table(config)
        routing = self.generate_routing_documentation(config)

        # Build complete doc
        doc = f"""# {hostname} - Device Documentation

**Generated**: {overview['generated_at']}
**Device Role**: {overview['role']}
**Management IP**: {overview['management_ip']}

---

## Overview

**Routing Protocols**: {', '.join(overview['routing_protocols'])}
**Key Features**: {', '.join(overview['key_features'])}

{overview['notable_config']}

---

## Interfaces

{interfaces}

---

## Routing Configuration

{routing}

---

**Auto-generated on**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}
*This documentation is auto-generated from device configuration.*
"""

        print(f"✅ Documentation generated for {hostname}")
        return doc

    def get_usage_stats(self) -> Dict:
        """Get API usage statistics."""
        total_tokens = self.usage_stats["input_tokens"] + self.usage_stats["output_tokens"]
        # Rough estimate: $3/$15 per 1M tokens for Sonnet
        estimated_cost = (self.usage_stats["input_tokens"] * 3 + self.usage_stats["output_tokens"] * 15) / 1_000_000
        return {
            **self.usage_stats,
            "total_tokens": total_tokens,
            "estimated_cost_usd": estimated_cost
        }

print("✅ ConfigDocumentationGenerator class loaded")

## Test with Sample Configuration

Let's test the documentation generator with a real Cisco router configuration.

In [ ]:
# Sample Cisco IOS router configuration
sample_router_config = """
hostname router-core-01
!
! Management
interface Loopback0
 ip address 192.168.1.1 255.255.255.255
 description Core Router Loopback
!
! Upstream
interface GigabitEthernet0/0
 description Uplink to ISP Primary
 ip address 203.0.113.1 255.255.255.252
 no shutdown
!
! Datacenter Connection
interface GigabitEthernet0/1
 description Connection to Datacenter Core
 ip address 10.0.1.1 255.255.255.0
 no shutdown
!
! Branch Connection
interface GigabitEthernet0/2
 description Link to Branch Office Router
 ip address 10.0.2.1 255.255.255.0
 no shutdown
!
! OSPF Configuration
router ospf 1
 router-id 192.168.1.1
 network 10.0.0.0 0.0.255.255 area 0
 network 192.168.1.1 0.0.0.0 area 0
 default-information originate
!
! BGP Configuration
router bgp 65001
 bgp router-id 192.168.1.1
 neighbor 203.0.113.2 remote-as 65002
 neighbor 203.0.113.2 description ISP_PRIMARY
 !
 address-family ipv4
  neighbor 203.0.113.2 activate
  redistribute ospf 1 metric 100
 exit-address-family
!
! Access Control
ip access-list extended MANAGEMENT_ACCESS
 permit tcp 10.0.0.0 0.0.255.255 any eq 22
 deny ip any any log
!
! Management Access
line vty 0 4
 access-class MANAGEMENT_ACCESS in
 transport input ssh
 logging synchronous
!
! Logging
logging buffered 5000
logging trap notifications
!"""

print("📋 Sample router configuration ready")
print(f"Config size: {len(sample_router_config)} characters")

In [ ]:
# Initialize generator and create documentation
generator = ConfigDocumentationGenerator(api_key=os.environ['ANTHROPIC_API_KEY'])

# Generate complete documentation
documentation = generator.generate_complete_documentation(
    config=sample_router_config,
    hostname="router-core-01"
)

# Display the generated documentation
print("\n" + "="*80)
print("GENERATED DOCUMENTATION")
print("="*80 + "\n")
print(documentation)

# Show API usage
stats = generator.get_usage_stats()
print("\n" + "="*80)
print("API USAGE STATS")
print("="*80)
print(f"Input tokens: {stats['input_tokens']}")
print(f"Output tokens: {stats['output_tokens']}")
print(f"Total tokens: {stats['total_tokens']}")
print(f"Estimated cost: ${stats['estimated_cost_usd']:.4f}")

## Part 2: Network Topology Extraction

Extract network topology from CDP/LLDP data and create visual diagrams.

In [ ]:
class NetworkTopologyDiagrammer:
    """Generate network diagrams from CDP/LLDP data."""

    def __init__(self, api_key: str):
        self.client = Anthropic(api_key=api_key)

    def extract_neighbors_from_cdp(self, cdp_output: str) -> List[Dict]:
        """Extract neighbor information from CDP/LLDP output."""

        prompt = f"""Extract neighbor information from this CDP/LLDP output.

Output:
{cdp_output}

Return JSON array of neighbors:
[
    {{
        "local_device": "this device name",
        "local_interface": "interface on this device",
        "remote_device": "neighbor device name",
        "remote_interface": "interface on neighbor",
        "platform": "device platform/model"
    }}
]

JSON:"""

        response = self.client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=2000,
            temperature=0,
            messages=[{"role": "user", "content": prompt}]
        )

        return json.loads(response.content[0].text)

    def generate_mermaid_diagram(
        self,
        devices: List[str],
        connections: List[Dict]
    ) -> str:
        """Generate Mermaid diagram syntax from topology data."""

        devices_str = ", ".join(devices)
        connections_str = "\n".join([
            f"- {c['local_device']} ({c['local_interface']}) <--> "
            f"{c['remote_device']} ({c['remote_interface']})"
            for c in connections
        ])

        prompt = f"""Create a network topology diagram using Mermaid syntax.

Devices:
{devices_str}

Connections:
{connections_str}

Generate Mermaid flowchart syntax showing:
1. All devices as nodes
2. All connections between them
3. Interface labels on links
4. Use shapes: R for routers, S for switches

Mermaid syntax:"""

        response = self.client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1500,
            temperature=0,
            messages=[{"role": "user", "content": prompt}]
        )

        return response.content[0].text

print("✅ NetworkTopologyDiagrammer class loaded")

In [ ]:
# Sample CDP output from multiple devices
cdp_data = {
    "router-core-01": """
Device ID: switch-dist-01
Interface: GigabitEthernet0/1, Port ID: GigabitEthernet1/0/1
Platform: cisco WS-C3850
Software Version: IOS XE 16.12

Device ID: router-core-02
Interface: GigabitEthernet0/2, Port ID: GigabitEthernet0/2
Platform: Cisco 4451-X
Software Version: IOS XE 16.12
    """,
    "switch-dist-01": """
Device ID: router-core-01
Interface: GigabitEthernet1/0/1, Port ID: GigabitEthernet0/1
Platform: Cisco 4451-X
Software Version: IOS XE 16.12

Device ID: switch-access-01
Interface: GigabitEthernet1/0/10, Port ID: GigabitEthernet1/0/1
Platform: cisco WS-C2960X-48TS
Software Version: IOS 15.2
    """
}

# Extract topology
print("🔍 Extracting network topology...")
diagrammer = NetworkTopologyDiagrammer(api_key=os.environ['ANTHROPIC_API_KEY'])

all_neighbors = []
all_devices = set()

for device_name, cdp_output in cdp_data.items():
    neighbors = diagrammer.extract_neighbors_from_cdp(cdp_output)
    all_neighbors.extend(neighbors)
    all_devices.add(device_name)
    for n in neighbors:
        all_devices.add(n['remote_device'])

print(f"✅ Found {len(all_devices)} devices, {len(all_neighbors)} connections")
print(f"\nDevices: {', '.join(sorted(all_devices))}")
print(f"\nConnections:")
for conn in all_neighbors:
    print(f"  • {conn['local_device']} ({conn['local_interface']}) <--> {conn['remote_device']} ({conn['remote_interface']})")

In [ ]:
# Generate Mermaid diagram
print("📊 Generating network topology diagram...")
mermaid_diagram = diagrammer.generate_mermaid_diagram(
    devices=list(all_devices),
    connections=all_neighbors
)

print("\n" + "="*80)
print("NETWORK TOPOLOGY DIAGRAM (Mermaid Format)")
print("="*80 + "\n")
print(mermaid_diagram)

print("\n💡 Copy this diagram to a Markdown file or GitHub to render it!")

## Best Practices for Production

### 1. Automate Documentation Generation
- Schedule daily generation during off-peak hours
- Run immediately after config changes
- Version control all documentation in Git

### 2. Cost Optimization
- Use Haiku for simple extraction (10x cheaper)
- Use Sonnet for complex analysis
- Cache unchanged configs
- Monitor API usage monthly

### 3. Quality Assurance
- Review first generation manually
- Validate extracted data against configs
- Use before/after comparison for changes
- Include confidence scores where appropriate

### 4. Security
- Sanitize configs before sending to API (remove passwords)
- Don't include sensitive data in public documentation
- Use separate API keys for production
- Audit all documentation access

### 5. Scalability
- Process devices in batches
- Add rate limiting between API calls
- Use background job queue for large networks
- Archive old documentation

### 6. Integration
- Integrate with CI/CD pipelines
- Trigger on config commits
- Send alerts for documentation failures
- Publish to internal wiki automatically

## Try It Yourself

Paste your own device configuration below and generate documentation for it!

In [ ]:
# Paste your device configuration here
your_config = """
# Paste your network device configuration here
# Can be Cisco IOS, NX-OS, Juniper, Arista, etc.
"""

your_hostname = "your-device-name"  # Change this to your device name

if your_config.strip() and "Paste your" not in your_config:
    print(f"Generating documentation for {your_hostname}...")
    your_generator = ConfigDocumentationGenerator(api_key=os.environ['ANTHROPIC_API_KEY'])
    your_doc = your_generator.generate_complete_documentation(
        config=your_config,
        hostname=your_hostname
    )
    print("\n" + "="*80)
    print(your_doc)
    print("="*80)
else:
    print("📝 Edit the cell above and paste your actual device configuration!")

## Summary

### What You've Learned

✅ **Auto-generate documentation** from device configs  
✅ **Extract structured data** (interfaces, routing, security)  
✅ **Create topology diagrams** using Mermaid format  
✅ **Analyze and validate** configurations  
✅ **Track API costs** and optimize usage  
✅ **Deploy to production** with proper error handling  

### Key Takeaways

1. **Documentation should be generated, not written**
2. **AI excels at synthesizing complex configurations**
3. **Automation scales to hundreds of devices**
4. **Version control provides audit trail**
5. **Small upfront investment saves massive time**

### Next Steps

1. **Clone the repository**: Get the full code examples
2. **Configure your network**: Export device configs
3. **Test locally**: Run on a few devices first
4. **Schedule generation**: Set up daily automation
5. **Integrate CI/CD**: Add to your deployment pipeline

### Resources

📚 [Chapter 13: Network Documentation Basics](https://github.com/eduardd76/AI_for_networking_and_security_engineers/blob/master/Volume-2-Practical-Applications/Chapters/Chapter-13-Network-Documentation-Basics.md)  
💻 [Source Code](https://github.com/eduardd76/AI_for_networking_and_security_engineers/tree/master/Volume-2-Practical-Applications/Code/Chapter-13-Network-Documentation-Basics)  
📖 [Full Book](https://github.com/eduardd76/AI_for_networking_and_security_engineers)  

---

**Happy documenting! 🚀**